# Fitting a FLMP
In this notebook, we will fit a Fuzzy Logical Model of Perception (FLMP) to audiovisual speech perception data.

In [1]:
import numpy as np
from os import path
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from scipy.special import comb
from scipy.stats import binom
from scipy.optimize import minimize, LinearConstraint, Bounds

## Prepare Data
The data consists of five text files, each containing seven rows and five columns, where:
- Row 1: Audiotorial data
- Row 2: Visual data
- Rows 3-7: Audiovisual data
    - a combination of rows 1 and 2
    - visual goes from 'b' (row 3) to 'd' (row 7) 
    - audio goes from 'b' (col 1) to 'd' (col 5)

In [2]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# split into the three different data types
# data_A = data[:, 0, :]
# data_V = data[:, 1, :]
# data_AV = data[:, 2:, :]

## Fit FLMP

In [3]:
# create softmax function
def baseline_softmax(x):
    x = np.concatenate([
        np.array(x).flatten(), [0]
    ])
    e = np.exp(x)
    return e / e.sum()

# create our own pmf function
def binomial_pmf(k, n, p):
    return comb(n, k) * np.power(p, k) * np.power(1 - p, n - k)

In [4]:
def objective_function(theta, data, n_samples=24):
    # extract audio and visual parameters
    theta_A = theta[0:(K-1)]
    theta_V = theta[(K-1): ]

    # get probabilities for audio and visual
    p_A = baseline_softmax(theta_A).reshape(-1,1)
    p_V = baseline_softmax(theta_V).reshape(-1,1)
    
    # compute audiovisual probabilities by
    # taking the outer product for all combinations of audio and visual
    p_AV = (p_A @ p_V.T) / (p_A @ p_V.T + (1 - p_A) @ (1 - p_V).T)
    
    # compute the log-likelihoods
    # L = np.log(binomial_pmf(data, n_samples, np.vstack([p_A.T, p_V.T, p_AV]))).sum()
    L_A = binom.logpmf(data[0], n_samples, p_A).flatten()
    L_V = binom.logpmf(data[1], n_samples, p_V).flatten()
    L_AV = binom.logpmf(data[2:], n_samples, p_AV).flatten()
    L = np.concatenate([L_A, L_V, L_AV]).sum()

    # return the negative log-likelihood
    return -L

In [5]:
def flmp_fit(data, n_samples):
    """Perform FLMP fit to data"""
    theta = np.zeros((K-1)*2)
    opt_result = minimize(objective_function, theta, args=(data, n_samples))
    objective, theta_A, theta_V = (
        opt_result.fun, 
        (opt_result.x[0:(K-1)]), 
        (opt_result.x[(K-1):])
    )
    return objective, theta_A, theta_V

In [6]:
# perform FLMP fit for each subject
theta_A = np.zeros((K, N))
theta_V = np.zeros((K, N))
neg_L = np.zeros(K)
for i in range(K):
    obj, tA, tV = flmp_fit(data[i], n_samples)

    # save results
    neg_L[i] = obj
    theta_A[i, :-1] = tA
    theta_V[i, :-1] = tV

    # print results
    print(f'\nResults for subject {i+1}')
    print(f'Negative log-likelihood: {neg_L[i]}')
    print("Audio:", theta_A[i])
    print("Visual:", theta_V[i])


Results for subject 1
Negative log-likelihood: 1991.0090793646602
Audio: [-0.83298402 -0.39242427 -0.11051291 -0.02504735  0.        ]
Visual: [-0.51342365 -0.36048932 -0.26596297 -0.04115978  0.        ]

Results for subject 2
Negative log-likelihood: 2163.470630921128
Audio: [-1.04381979 -0.39016762 -0.14588921 -0.02251161  0.        ]
Visual: [-0.39810196 -0.22548924 -0.13804517 -0.0679782   0.        ]

Results for subject 3
Negative log-likelihood: 1851.3917708038994
Audio: [-0.50185151 -0.31734084 -0.13454004 -0.04115398  0.        ]
Visual: [-0.69550475 -0.57201194 -0.35936676 -0.07644569  0.        ]

Results for subject 4
Negative log-likelihood: 1269.771788408241
Audio: [-0.44062118 -0.33439257 -0.25944566 -0.15329477  0.        ]
Visual: [-1.24479319 -1.12777827 -0.94946008 -0.18509925  0.        ]

Results for subject 5
Negative log-likelihood: 1764.0995226579707
Audio: [-0.42824501 -0.22158146 -0.14804333 -0.05783815  0.        ]
Visual: [-1.00784283 -0.89023368 -0.490687